# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('Churn').getOrCreate()

In [5]:
data = spark.read.csv('/FileStore/tables/customer_churn.csv', inferSchema=True, header=True)

In [6]:
data.printSchema()

root
-- Names: string (nullable = true)
-- Age: double (nullable = true)
-- Total_Purchase: double (nullable = true)
-- Account_Manager: integer (nullable = true)
-- Years: double (nullable = true)
-- Num_Sites: double (nullable = true)
-- Onboard_date: string (nullable = true)
-- Location: string (nullable = true)
-- Company: string (nullable = true)
-- Churn: integer (nullable = true)

In [7]:
data.show()

+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
 Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites| Onboard_date| Location| Company|Churn|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
 Cameron Williams|42.0| 11066.8| 0| 7.22| 8.0|2013-08-30 07:00:40|10265 Elizabeth M...| Harvey LLC| 1|
 Kevin Mueller|41.0| 11916.22| 0| 6.5| 11.0|2013-08-13 00:38:46|6157 Frank Garden...| Wilson PLC| 1|
 Eric Lozano|38.0| 12884.75| 0| 6.67| 12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...| 1|
 Phillip White|42.0| 8010.76| 0| 6.71| 10.0|2014-04-22 12:43:12|13120 Daniel Moun...| Smith Inc| 1|
 Cynthia Norton|37.0| 9191.58| 0| 5.56| 9.0|2016-01-19 15:31:15|765 Tricia Row Ka...| Love-Jones| 1|
 Jessica Williams|48.0| 10356.02| 0| 5.12| 8.0|2009-03-03 23:13:37|6187 Olson Mounta...| Kelly-Warren| 1|
 Eric Butler|44.0| 11331.58| 1| 5.23| 11.0|2016-12-05 03:35:43|4846 Savannah Roa...| Reynolds-Sheppard| 1|
 Zachary Walsh|32.0| 9885.12| 1| 6.92| 9.0|2006-03-09 14:50:20|25271 Roy Express...| Singh-Cole| 1|
 Ashlee Carr|43.0| 14062.6| 1| 5.46| 11.0|2011-09-29 05:47:23|3725 Caroline Str...| Lopez PLC| 1|
 Jennifer Lynch|40.0| 8066.94| 1| 7.11| 11.0|2006-03-28 15:42:45|363 Sandra Lodge ...| Reed-Martinez| 1|
 Paula Harris|30.0| 11575.37| 1| 5.22| 8.0|2016-11-13 13:13:01|Unit 8120 Box 916...|Briggs, Lamb and ...| 1|
 Bruce Phillips|45.0| 8771.02| 1| 6.64| 11.0|2015-05-28 12:14:03|Unit 1895 Box 094...| Figueroa-Maynard| 1|
 Craig Garner|45.0| 8988.67| 1| 4.84| 11.0|2011-02-16 08:10:47|897 Kelley Overpa...| Abbott-Thompson| 1|
 Nicole Olson|40.0| 8283.32| 1| 5.1| 13.0|2012-11-22 05:35:03|11488 Weaver Cape...|Smith, Kim and Ma...| 1|
 Harold Griffin|41.0| 6569.87| 1| 4.3| 11.0|2015-03-28 02:13:44|1774 Peter Row Ap...|Snyder, Lee and M...| 1|
 James Wright|38.0| 10494.82| 1| 6.81| 12.0|2015-07-22 08:38:40|45408 David Path ...| Sanders-Pierce| 1|
 Doris Wilkins|45.0| 8213.41| 1| 7.35| 11.0|2006-09-03 06:13:55|28216 Wright Moun...|Andrews, Adams an...| 1|
Katherine Carpenter|43.0| 11226.88| 0| 8.08| 12.0|2006-10-22 04:42:38|Unit 4948 Box 481...|Morgan, Phillips ...| 1|
 Lindsay Martin|53.0| 5515.09| 0| 6.85| 8.0|2015-10-07 00:27:10|69203 Crosby Divi...| Villanueva LLC| 1|
 Kathy Curry|46.0| 8046.4| 1| 5.69| 8.0|2014-11-06 23:47:14|9569 Caldwell Cre...|Berry, Orr and Ca...| 1|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
only showing top 20 rows

In [8]:
data.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
summary| Names| Age| Total_Purchase| Account_Manager| Years| Num_Sites| Onboard_date| Location| Company| Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
 count| 900| 900| 900| 900| 900| 900| 900| 900| 900| 900|
 mean| null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777| null| null| null|0.16666666666666666|
 stddev| null|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.7648355920350969| null| null| null| 0.3728852122772358|
 min| Aaron King| 22.0| 100.0| 0| 1.0| 3.0|2006-01-02 04:16:13|00103 Jeffrey Cre...| Abbott-Thompson| 0|
 max|Zachary Walsh| 65.0| 18026.01| 1| 9.15| 14.0|2016-12-28 04:07:38|Unit 9800 Box 287...|Zuniga, Clark and...| 1|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+

In [9]:
data.columns

Out[7]: ['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [10]:
#Check if the data contains any null or nan values
from pyspark.sql.functions import isnan, when, count, col, isnull

In [11]:
data.select([count(when(isnan(c), c)).alias(c) for c in data.columns]).show()

+-----+---+--------------+---------------+-----+---------+------------+--------+-------+-----+
Names|Age|Total_Purchase|Account_Manager|Years|Num_Sites|Onboard_date|Location|Company|Churn|
+-----+---+--------------+---------------+-----+---------+------------+--------+-------+-----+
 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
+-----+---+--------------+---------------+-----+---------+------------+--------+-------+-----+

In [12]:
data.select([count(when(isnull(c), c)).alias(c) for c in data.columns]).show()

+-----+---+--------------+---------------+-----+---------+------------+--------+-------+-----+
Names|Age|Total_Purchase|Account_Manager|Years|Num_Sites|Onboard_date|Location|Company|Churn|
+-----+---+--------------+---------------+-----+---------+------------+--------+-------+-----+
 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
+-----+---+--------------+---------------+-----+---------+------------+--------+-------+-----+

In [13]:
#Select relevant columns for the model:

my_col = data.select(['Age',
 'Total_Purchase',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Churn'])

In [14]:
from pyspark.ml.feature import (VectorAssembler,VectorIndexer,
                                OneHotEncoder,StringIndexer)

In [15]:
#Encode 'Onboard date' column
Onboard_date_index = StringIndexer(inputCol='Onboard_date', outputCol='Onboard_date_index')
indexed = Onboard_date_index.fit(my_col).transform(my_col)
indexed.columns

In [16]:
#Assemble all the features into vectors
assembled = VectorAssembler(inputCols=['Age',
 'Total_Purchase',
 'Years',
 'Num_Sites',
 'Onboard_date_index'],
 outputCol='features')

In [17]:
#Generate final datatable
output = assembled.transform(indexed)
final_data = output.select('features','Churn')
final_data.show()

+--------------------+-----+
 features|Churn|
+--------------------+-----+
[42.0,11066.8,7.2...| 1|
[41.0,11916.22,6....| 1|
[38.0,12884.75,6....| 1|
[42.0,8010.76,6.7...| 1|
[37.0,9191.58,5.5...| 1|
[48.0,10356.02,5....| 1|
[44.0,11331.58,5....| 1|
[32.0,9885.12,6.9...| 1|
[43.0,14062.6,5.4...| 1|
[40.0,8066.94,7.1...| 1|
[30.0,11575.37,5....| 1|
[45.0,8771.02,6.6...| 1|
[45.0,8988.67,4.8...| 1|
[40.0,8283.32,5.1...| 1|
[41.0,6569.87,4.3...| 1|
[38.0,10494.82,6....| 1|
[45.0,8213.41,7.3...| 1|
[43.0,11226.88,8....| 1|
[53.0,5515.09,6.8...| 1|
[46.0,8046.4,5.69...| 1|
+--------------------+-----+
only showing top 20 rows

In [18]:
#Split the dataset into train and test
train_data, test_data = final_data.randomSplit([0.7,.3])

In [19]:
from pyspark.ml.classification import LogisticRegression

In [20]:
lr = LogisticRegression(featuresCol='features',labelCol='Churn')
#Fit the model on the training set
lr_model = lr.fit(train_data)

In [21]:
#Evaluate the model on the test set
results = lr_model.transform(test_data)
results.show()

+--------------------+-----+--------------------+--------------------+----------+
 features|Churn| rawPrediction| probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
[22.0,11254.38,4....| 0|[5.18559438044620...|[0.99443453860049...| 0.0|
[25.0,9672.03,5.4...| 0|[4.59252585384431...|[0.98997428676813...| 0.0|
[28.0,11128.95,5....| 0|[4.62389569752913...|[0.99028090075905...| 0.0|
[29.0,11274.46,4....| 0|[5.06490332207507...|[0.99372510163158...| 0.0|
[29.0,13255.05,4....| 0|[4.71567590802735...|[0.99112564721933...| 0.0|
[30.0,6744.87,5.1...| 0|[3.43983336909837...|[0.96892649931942...| 0.0|
[30.0,7960.64,2.7...| 1|[3.84709095143567...|[0.97910422171267...| 0.0|
[30.0,8403.78,4.1...| 0|[6.40980732490722...|[0.99835736121388...| 0.0|
[30.0,10744.14,7....| 1|[1.90148058749008...|[0.87005900716000...| 0.0|
[31.0,5304.6,5.29...| 0|[3.24969226517223...|[0.96266205306390...| 0.0|
[31.0,5387.75,6.8...| 0|[2.20611949018238...|[0.90079769871249...| 0.0|
[31.0,10058.87,6....| 0|[4.71494127917469...|[0.99111918338696...| 0.0|
[31.0,11297.57,6....| 1|[1.29849081783951...|[0.78558088059054...| 0.0|
[31.0,12264.68,5....| 0|[3.91240197951449...|[0.98039944069780...| 0.0|
[32.0,8011.38,5.3...| 0|[1.90322745541868...|[0.87025637400229...| 0.0|
[32.0,8617.98,6.2...| 1|[1.25633723857600...|[0.77839493944571...| 0.0|
[32.0,9036.27,7.1...| 0|[-0.5768360972460...|[0.35966093211436...| 1.0|
[32.0,11540.86,3....| 0|[6.96479704674366...|[0.99905633672489...| 0.0|
[32.0,12479.72,4....| 0|[4.66588105473418...|[0.99067678731139...| 0.0|
[32.0,13630.93,4....| 0|[2.39040855686782...|[0.91609297783273...| 0.0|
+--------------------+-----+--------------------+--------------------+----------+
only showing top 20 rows

In [22]:
results.select('Churn','prediction').show()

+-----+----------+
Churn|prediction|
+-----+----------+
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 1| 0.0|
 0| 0.0|
 1| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 1| 0.0|
 0| 0.0|
 0| 0.0|
 1| 0.0|
 0| 1.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
+-----+----------+
only showing top 20 rows

In [23]:
training_summary = lr_model.summary

In [24]:
training_summary.predictions.describe().show()

+-------+-------------------+-------------------+
summary| Churn| prediction|
+-------+-------------------+-------------------+
 count| 633| 633|
 mean|0.16429699842022116|0.12480252764612954|
 stddev| 0.3708378986773153|0.33075623054484365|
 min| 0.0| 0.0|
 max| 1.0| 1.0|
+-------+-------------------+-------------------+

In [25]:
pred_and_labels = lr_model.evaluate(test_data)
pred_and_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
 features|Churn| rawPrediction| probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
[22.0,11254.38,4....| 0|[5.18559438044620...|[0.99443453860049...| 0.0|
[25.0,9672.03,5.4...| 0|[4.59252585384431...|[0.98997428676813...| 0.0|
[28.0,11128.95,5....| 0|[4.62389569752913...|[0.99028090075905...| 0.0|
[29.0,11274.46,4....| 0|[5.06490332207507...|[0.99372510163158...| 0.0|
[29.0,13255.05,4....| 0|[4.71567590802735...|[0.99112564721933...| 0.0|
[30.0,6744.87,5.1...| 0|[3.43983336909837...|[0.96892649931942...| 0.0|
[30.0,7960.64,2.7...| 1|[3.84709095143567...|[0.97910422171267...| 0.0|
[30.0,8403.78,4.1...| 0|[6.40980732490722...|[0.99835736121388...| 0.0|
[30.0,10744.14,7....| 1|[1.90148058749008...|[0.87005900716000...| 0.0|
[31.0,5304.6,5.29...| 0|[3.24969226517223...|[0.96266205306390...| 0.0|
[31.0,5387.75,6.8...| 0|[2.20611949018238...|[0.90079769871249...| 0.0|
[31.0,10058.87,6....| 0|[4.71494127917469...|[0.99111918338696...| 0.0|
[31.0,11297.57,6....| 1|[1.29849081783951...|[0.78558088059054...| 0.0|
[31.0,12264.68,5....| 0|[3.91240197951449...|[0.98039944069780...| 0.0|
[32.0,8011.38,5.3...| 0|[1.90322745541868...|[0.87025637400229...| 0.0|
[32.0,8617.98,6.2...| 1|[1.25633723857600...|[0.77839493944571...| 0.0|
[32.0,9036.27,7.1...| 0|[-0.5768360972460...|[0.35966093211436...| 1.0|
[32.0,11540.86,3....| 0|[6.96479704674366...|[0.99905633672489...| 0.0|
[32.0,12479.72,4....| 0|[4.66588105473418...|[0.99067678731139...| 0.0|
[32.0,13630.93,4....| 0|[2.39040855686782...|[0.91609297783273...| 0.0|
+--------------------+-----+--------------------+--------------------+----------+
only showing top 20 rows

In [27]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [28]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                       labelCol='Churn')

In [29]:
AUC = my_eval.evaluate(results)

In [30]:
AUC

Out[41]: 0.7513771394845564

## Use of pipelines

In [32]:
from pyspark.ml import Pipeline

In [33]:
log_reg_churn = LogisticRegression(featuresCol='features',labelCol='Churn')

In [34]:
pipeline = Pipeline(stages=[Onboard_date_index,
                           assembled,log_reg_churn])

In [35]:
train_data_churn, test_data_churn = data.randomSplit([0.7,0.3])

In [36]:
fit_model = pipeline.fit(train_data_churn)

In [37]:
results_churn = fit_model.transform(test_data_churn)